In [5]:
import argparse
import os
import re
import traceback
import pandas as pd
import importlib
from src.pipeline.heuristic_evolver import HeuristicEvolver
from src.util.util import search_file
from src.util.llm_client.get_llm_client import get_llm_client

# ----------------------
# 在下方设置参数（对应命令行参数）
# ----------------------
problem = "tsp"  # 替换为具体问题名称，如 "max_cut"、"tsp" 等
seed_heuristic = "cheapest_insertion_8fc1.py"  # 初始启发式文件路径或名称
smoke_test = False  # 是否进行冒烟测试
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  # LLM配置文件路径
evolution_dir = "evolution_data"  # 进化数据集目录
validation_dir = None  # 验证数据集目录（默认使用进化数据集目录）
perturbation_heuristic = None  # 扰动启发式文件（可选）
perturbation_ratio = 0.1  # 扰动比例
perturbation_time = 1000  # 最大扰动尝试次数
max_refinement_round = 5  # 最大优化轮数
filter_num = 1  # 保留的顶级启发式数量
evolution_rounds = 3  # 进化迭代次数

# ----------------------
# 以下为核心执行代码，无需修改
# ----------------------
def main():
    try:
        # 处理参数默认值
        validation_dir = validation_dir if validation_dir else evolution_dir
        
        # 搜索问题相关路径
        evolution_dir_resolved = search_file(evolution_dir, problem)
        validation_dir_resolved = search_file(validation_dir, problem)
        
        # 处理初始启发式文件
        if not seed_heuristic.endswith(".py"):
            seed_heuristic_file = seed_heuristic + ".py"
        else:
            seed_heuristic_file = seed_heuristic
        seed_heuristic_file = search_file(seed_heuristic_file, problem)
        if not seed_heuristic_file:
            raise FileNotFoundError(f"未找到初始启发式文件: {seed_heuristic}")
        
        # 处理扰动启发式文件
        if perturbation_heuristic is None:
            # 自动查找随机扰动启发式
            heuristic_dir = os.path.join("src", "problems", problem, "heuristics", "basic_heuristics")
            if not os.path.exists(heuristic_dir):
                raise FileNotFoundError(f"启发式目录不存在: {heuristic_dir}")
            perturbation_candidates = [
                f for f in os.listdir(heuristic_dir) 
                if re.match(r"random_....\.py", f)
            ]
            if not perturbation_candidates:
                raise Exception("未找到默认扰动启发式文件（格式为 random_xxxx.py）")
            perturbation_heuristic_file = os.path.join(heuristic_dir, perturbation_candidates[0])
        else:
            if not perturbation_heuristic.endswith(".py"):
                perturbation_heuristic_file = perturbation_heuristic + ".py"
            else:
                perturbation_heuristic_file = perturbation_heuristic
            perturbation_heuristic_file = search_file(perturbation_heuristic_file, problem)
            if not perturbation_heuristic_file:
                raise FileNotFoundError(f"未找到扰动启发式文件: {perturbation_heuristic}")
        
        # 初始化LLM客户端
        prompt_dir = os.path.join("src", "problems", "base", "prompt")
        llm_client = get_llm_client(llm_config_file, prompt_dir, None)
        
        # 初始化启发式进化器并执行进化
        heuristic_evolver = HeuristicEvolver(
            llm_client=llm_client,
            problem=problem,
            evolution_dir=evolution_dir_resolved,
            validation_dir=validation_dir_resolved
        )
        
        print("开始启发式进化...")
        evolved_heuristics = heuristic_evolver.evolve(
            basic_heuristic_file=seed_heuristic_file,
            perturbation_heuristic_file=perturbation_heuristic_file,
            perturbation_ratio=perturbation_ratio,
            perturbation_time=perturbation_time,
            max_refinement_round=max_refinement_round,
            filtered_num=filter_num,
            evolution_round=evolution_rounds,
            smoke_test=smoke_test
        )
        
        print("\n进化完成！结果如下:")
        for h in evolved_heuristics:
            print(f"启发式文件: {h[0]}, 性能分数: {h[1]}")
            
    except Exception as e:
        print(f"执行出错: {str(e)}")
        traceback.print_exc()

# 执行主函数
if __name__ == "__main__":
    main()

执行出错: cannot access local variable 'validation_dir' where it is not associated with a value


Traceback (most recent call last):
  File "C:\Users\gongh\AppData\Local\Temp\ipykernel_9500\77669974.py", line 33, in main
    validation_dir = validation_dir if validation_dir else evolution_dir
                                       ^^^^^^^^^^^^^^
UnboundLocalError: cannot access local variable 'validation_dir' where it is not associated with a value
